### Convert Given array as a sparse connected fcn

In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
# 0: not connected
# 1: linear
# 2: ReLU
# 3: sigmoid

activations = [None, None, nn.ReLU(), nn.Sigmoid()]    

In [28]:
class MatrixForWANN():
    
    def __init__(self, mat, in_dim, out_dim):
        # get when initialized
        self.mat = mat  
        self.in_dim = in_dim  
        self.out_dim = out_dim  
        
        #calculate
        self.num_hidden_nodes = self.mat.shape[1] - self.in_dim   
        
        #when matrix has hidden layer  
        if self.num_hidden_nodes == 1:  
            self.hidden_dim = [1] 
        elif self.num_hidden_nodes == 0:
            self.hidden_dim = []
        else:
            self.hidden_dim = self.get_hidden_dim()   
            
            
    def get_hidden_dim(self):
        in_dim = self.in_dim
        out_dim = self.out_dim
        mat_mask = self.mat
        
        hidden_dim_list = []
        start_col_idx = 0
        finish_col_idx = in_dim -1   
        
        while(True):
            
            if finish_col_idx >= mat_mask.shape[1]:   
                print(finish_col_idx)
                break  
            
            if ((mat_mask.shape[0] - sum(hidden_dim_list)) == out_dim):  #example4 해결
                 break  #지금 hidden dimension들 합이랑 output dim 합이 row길이랑 같으면 더이상 탐색 필요 x
            
            for i in range(sum(hidden_dim_list), len(mat_mask)): #이부분이상한데..?   
    
                #밑에처럼 하면 example 2에서 오류가 남.
                #skip connection에 대한 예외처리 해줘야 함   
    
                if(mat_mask[i,start_col_idx:(finish_col_idx + 1)].sum() == 0):   
                
                    hidden_dim = i - sum(hidden_dim_list)
                    hidden_dim_list += [hidden_dim]
                    start_col_idx = finish_col_idx + 1
                    finish_col_idx += hidden_dim   
                    break    
                    
        return hidden_dim_list     
    
    
    

In [31]:
mat4 = np.array([[3,0,1,0,0],   
                [0,3,0,0,0],     
                [0,0,0,2,0],  
                [0,0,0,0,2],   
                [0,0,1,2,0],   
                [0,0,1,0,0]])    
in_dim = 3   
out_dim = 4  
mat_wann4 = MatrixForWANN(mat4, in_dim, out_dim)    

In [33]:
mat_wann4.hidden_dim   

[2]

In [34]:
mat5 = np.array([[0,2,0,0,2,0,0,0,0,0],  
                [2,0,2,0,0,0,0,0,0,0],  
                [0,2,0,2,0,0,0,0,0,0],  
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,2,0,0,1,1,0,0], 
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])   
in_dim = 5   
out_dim = 2   
mat_wann5 = MatrixForWANN(mat5, in_dim, out_dim)    

In [35]:
mat_wann5.hidden_dim   

[3, 2]

In [36]:
mat3 = np.array([[2,0,0,3,0],
                [0,0,0,0,1]])  
in_dim = 5  
out_dim = 2  
mat_wann3 = MatrixForWANN(mat3, in_dim, out_dim)   


In [37]:
mat_wann3.hidden_dim  

[]

In [38]:
mat2 = np.array([[2,0,0,0,0,0],
                [0,0,0,3,0,1]])
in_dim = 5
out_dim = 1
mat_wann2 = MatrixForWANN(mat2, in_dim, out_dim)  

In [39]:
mat_wann2.hidden_dim

[1]

In [40]:
mat1 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],  
                [0,0,0,0,0,0,0,0,3,0]])
in_dim = 5
out_dim = 2
mat_wann1 = MatrixForWANN(mat1, in_dim, out_dim)

In [42]:
mat_wann1.hidden_dim  

[3, 2]

## Class WANNFCN

In [44]:
def wrap_activation(x, idx_activation, activations):   
    if idx_activation == 0:
        assert True
    elif idx_activation == 1:
        return nn.Linear(1,1)(x)
    else:
        return activations[idx_activation](nn.Linear(1,1)(x))

In [ ]:
class WANNFCN(nn.Module):
    
    def __init__(self, mat_wann, activations):  
        super(WANNFCN, self).__init__()
        self.mat = mat_wann.mat
        self.in_dim = mat_wann.in_dim
        self.out_dim = mat_wann.out_dim  
        self.num_hidden_nodes = mat_wann.num_hidden_nodes
        self.hidden_dim = mat_wann.hidden_dim
        
        self.activations = activations
        
        self.nodes = {} #node 담는 딕셔너리
    
    
    def to_hidden(self, x):
        
        hidden_node_counts = 0
        
        for idx_hidden_layer, num_hidden_nodes in enumerate(self.hidden_dim): #각 hidden layer와 그에 해당하는 dimension
            
            #First hidden layer
            if idx_hidden_layer == 0:
                indices_hidden_row = list(range(0, num_hidden_nodes))
                for idx_hidden_row in indices_hidden_row: #각 hidden layer마다 hidden node list 
                    connections_from_input = self.mat[idx_hidden_row, :self.in_dim] #matrix 잘라옴 (한 hidden node에 대해서)
                    if connections_from_input.sum() != 0:  
                        count_connection = 0
                        input_node = None  
                        
                        for idx_input_col, activation_type in enumerate(connections_from_input):
                            if activation_type != 0 and count_connection == 0:
                                input_node = wrap_activation(x[:, idx_input_col].view(-1,1), activation_type, activations)  
                                count_connection += 1
                            elif activation_type !=0 and count_connection != 0:
                                new_node = wrap_activation(x[:, idx_input_col].view(-1,1), activation_type, activations)
            
                    self.nodes['hidden_%d'%idx_hidden_row] = input_node     
            
            else: # other hidden layers
                indices_hidden_row = list()
                
                
  